- we have cleaned and explored the data
- now i'll be using cleaned dataset
- First spliting the data and as 60% training, 20% validation and 20% for testing and then
- I'll be using 5 fold cross-validation on 5 algorithms
- Logistic Regression
- Support vector machine(SVM)
- Multilayer perceptron(MLP)
- Random Forest
- Boosting

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('cleaned.csv')

In [5]:
df.head()

,Survived,Pclass,Sex,Age,Fare,Fam_cnt,cabin_ind
0,0,3,1,22.0,7.2500,1,0
1,1,1,0,38.0,71.2833,1,1
2,1,3,0,26.0,7.9250,0,0
3,1,1,0,35.0,53.1000,1,1
4,0,3,1,35.0,8.0500,0,0


# Spliting the data for train, test and validation

In [8]:
features = df.drop('Survived',axis=1)
label = df['Survived']

In [10]:
features.head()

,Pclass,Sex,Age,Fare,Fam_cnt,cabin_ind
0,3,1,22.0,7.2500,1,0
1,1,0,38.0,71.2833,1,1
2,3,0,26.0,7.9250,0,0
3,1,0,35.0,53.1000,1,1
4,3,1,35.0,8.0500,0,0


In [11]:
label.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [14]:
x_train,x_test,y_train,y_test = train_test_split(features,label,test_size=0.4,random_state=42)
x_val,x_test,y_val,y_test = train_test_split(x_test,y_test,test_size=0.5,random_state=42)

In [15]:
x_train.to_csv('train_features.csv',index=False)
y_train.to_csv('train_label.csv',index=False)
x_test.to_csv('test_features.csv',index=False)
y_test.to_csv('test_label.csv',index=False)
x_val.to_csv('val_features.csv',index=False)
y_val.to_csv('val_label.csv',index=False)

In [28]:
for dataset in [y_train, y_val, y_test]:
    print(round(len(dataset) / len(label),1))

0.6
0.2
0.2


In [23]:
for dataset in [y_train, y_val, y_test]:
    print(len(dataset))

534
178
179


# 1) Train the model in Logistic Regression

In [32]:
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

LR_model = LogisticRegression()

In [33]:
tr_feature = pd.read_csv('train_features.csv')
tr_label = pd.read_csv('train_label.csv')

In [35]:
def print_result(results):
    print(f'BEST PARAMS : {results.best_params_}')
    
    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    
    for mean,std,param in zip(means,stds,params):
        print(f'{round(mean,3)} (+/-{round(std *2,3)} for {param}')

In [37]:
parameters = {'C' : [0.001,0.01,0.1,1,10,100,1000]}
cv = GridSearchCV(LR_model,parameters,cv=5)
cv.fit(tr_feature,tr_label.values.ravel())
print_result(cv)

BEST PARAMS : {'C': 1}
0.67 (+/-0.077 for {'C': 0.001}
0.708 (+/-0.098 for {'C': 0.01}
0.777 (+/-0.134 for {'C': 0.1}
0.8 (+/-0.118 for {'C': 1}
0.794 (+/-0.116 for {'C': 10}
0.794 (+/-0.116 for {'C': 100}
0.794 (+/-0.116 for {'C': 1000}


In [38]:
cv.best_estimator_

LogisticRegression(C=1)

In [39]:
joblib.dump(cv.best_estimator_,'LR_MODEL.pkl')

['LR_MODEL.pkl']

# 2) Train the model in Support Vector Machine(SVM)

In [40]:
from sklearn.svm import SVC
svm_model = SVC()

In [43]:
parameters = {
    'C':[0.1,1,10],
    'kernel':['linear','rbf']}
cv = GridSearchCV(svm_model,parameters,cv=5)
cv.fit(tr_feature,tr_label.values.ravel())
print_result(cv)

BEST PARAMS : {'C': 0.1, 'kernel': 'linear'}
0.796 (+/-0.115 for {'C': 0.1, 'kernel': 'linear'}
0.654 (+/-0.06 for {'C': 0.1, 'kernel': 'rbf'}
0.796 (+/-0.115 for {'C': 1, 'kernel': 'linear'}
0.661 (+/-0.048 for {'C': 1, 'kernel': 'rbf'}
0.796 (+/-0.115 for {'C': 10, 'kernel': 'linear'}
0.684 (+/-0.07 for {'C': 10, 'kernel': 'rbf'}


In [50]:
joblib.dump(cv.best_estimator_,'svm_model.pkl')

['svm_model.pkl']

# 3) Train the model in Multilayer Perceptron(MLP)

In [51]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier()

In [46]:
parameters = {'hidden_layer_sizes' : [(10,),(50,),(100,)],
             'learning_rate': ['constant', 'invscaling', 'adaptive'],
             'activation': ['relu', 'tanh', 'logistic']}
cv = GridSearchCV(mlp_model,parameters,cv=5)
cv.fit(tr_feature,tr_label)
print_result(cv)

/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/

/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/

/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  retur

/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/

/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/

/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/

/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/

/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kali/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/

BEST PARAMS : {'activation': 'tanh', 'hidden_layer_sizes': (100,), 'learning_rate': 'invscaling'}
0.695 (+/-0.089 for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'constant'}
0.747 (+/-0.166 for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'invscaling'}
0.685 (+/-0.101 for {'activation': 'relu', 'hidden_layer_sizes': (10,), 'learning_rate': 'adaptive'}
0.768 (+/-0.093 for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'constant'}
0.77 (+/-0.117 for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'invscaling'}
0.773 (+/-0.117 for {'activation': 'relu', 'hidden_layer_sizes': (50,), 'learning_rate': 'adaptive'}
0.785 (+/-0.105 for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}
0.794 (+/-0.115 for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'invscaling'}
0.79 (+/-0.099 for {'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate'

/home/kali/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [52]:
joblib.dump(svm_model,'MLP_model.pkl')

['MLP_model.pkl']

# 4) Train the model in RandomForest

In [53]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()

In [55]:
parameters = {'n_estimators':[5,10,25,30,100,150,200],
             'max_depth': [2,4,8,16,32,64,None]}
cv = GridSearchCV(rf_model,parameters,cv=5)
cv.fit(tr_feature,tr_label.values.ravel())
print_result(cv)

BEST PARAMS : {'max_depth': 4, 'n_estimators': 100}
0.787 (+/-0.164 for {'max_depth': 2, 'n_estimators': 5}
0.798 (+/-0.099 for {'max_depth': 2, 'n_estimators': 10}
0.798 (+/-0.111 for {'max_depth': 2, 'n_estimators': 25}
0.792 (+/-0.097 for {'max_depth': 2, 'n_estimators': 30}
0.796 (+/-0.12 for {'max_depth': 2, 'n_estimators': 100}
0.803 (+/-0.113 for {'max_depth': 2, 'n_estimators': 150}
0.805 (+/-0.114 for {'max_depth': 2, 'n_estimators': 200}
0.813 (+/-0.111 for {'max_depth': 4, 'n_estimators': 5}
0.817 (+/-0.115 for {'max_depth': 4, 'n_estimators': 10}
0.824 (+/-0.111 for {'max_depth': 4, 'n_estimators': 25}
0.82 (+/-0.113 for {'max_depth': 4, 'n_estimators': 30}
0.826 (+/-0.109 for {'max_depth': 4, 'n_estimators': 100}
0.824 (+/-0.11 for {'max_depth': 4, 'n_estimators': 150}
0.822 (+/-0.103 for {'max_depth': 4, 'n_estimators': 200}
0.826 (+/-0.063 for {'max_depth': 8, 'n_estimators': 5}
0.824 (+/-0.077 for {'max_depth': 8, 'n_estimators': 10}
0.824 (+/-0.052 for {'max_depth': 8,

In [56]:
joblib.dump(cv.best_estimator_,'rf_model.pkl')

['rf_model.pkl']

# 5) Train the model in Boosting

In [57]:
from sklearn.ensemble import GradientBoostingClassifier

bst_model = GradientBoostingClassifier()

In [58]:
parameters = {'learning_rate':[0.001,0.01,0.1,1,5,10,50,100],
              'n_estimators':[5,10,50,100,150,200],
              'max_depth':[1,3,5,6,7,9]}
cv = GridSearchCV(bst_model,parameters,cv=5)
cv.fit(tr_feature,tr_label.values.ravel())
print_result(cv)

BEST PARAMS : {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 5}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 10}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 50}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 100}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 150}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 1, 'n_estimators': 200}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 5}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 10}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 50}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 100}
0.624 (+/-0.007 for {'learning_rate': 0.001, 'max_depth': 3, 'n_estimators': 150}
0.624 (+/-0.007 for {'learning_ra

In [60]:
joblib.dump(cv.best_estimator_,'bst_model.pkl')

['bst_model.pkl']